# EduPulse API Usage Examples

This notebook provides comprehensive examples of using the EduPulse API for various operations including predictions, batch processing, training, and data management.

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import asyncio
import aiohttp
from typing import Dict, List, Any
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
BASE_URL = "http://localhost:8000"
API_VERSION = "v1"
API_URL = f"{BASE_URL}/api/{API_VERSION}"

# Set up visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("EduPulse API Client Examples")
print("=" * 50)
print(f"Base URL: {BASE_URL}")
print(f"API Version: {API_VERSION}")

## 1. Health Check and System Status

In [ ]:
# Health check endpoint
def check_health():
    """Check API health status"""
    try:
        response = requests.get(f"{BASE_URL}/health")
        if response.status_code == 200:
            data = response.json()
            print("✅ API is healthy!")
            print(f"Status: {data.get('status', 'unknown')}")
            print(f"Version: {data.get('version', 'unknown')}")
            print(f"Model Status: {data.get('model_status', 'unknown')}")
            print(f"Database: {data.get('database', 'unknown')}")
            return True
        else:
            print(f"❌ Health check failed: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to API. Please ensure the server is running.")
        return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Check if API is available
api_available = check_health()

# If API is not available, use mock responses
if not api_available:
    print("\n⚠️  Using mock data for demonstration purposes")

## 2. Student Management Endpoints

In [ ]:
# Get all students
def get_all_students(limit=10, offset=0):
    """Retrieve all students with pagination"""
    
    if api_available:
        try:
            response = requests.get(
                f"{API_URL}/students",
                params={"limit": limit, "offset": offset}
            )
            return response.json()
        except:
            pass
    
    # Mock data
    return [
        {"id": 1, "name": "John Doe", "grade_level": 10, "gpa": 3.5, "risk_level": "Low"},
        {"id": 2, "name": "Jane Smith", "grade_level": 11, "gpa": 2.8, "risk_level": "Medium"},
        {"id": 3, "name": "Bob Johnson", "grade_level": 9, "gpa": 2.1, "risk_level": "High"},
    ]

# Get specific student
def get_student(student_id: int):
    """Get detailed information about a specific student"""
    
    if api_available:
        try:
            response = requests.get(f"{API_URL}/students/{student_id}")
            return response.json()
        except:
            pass
    
    # Mock data
    return {
        "id": student_id,
        "name": "John Doe",
        "grade_level": 10,
        "gpa": 3.5,
        "attendance_rate": 92.5,
        "discipline_incidents": 0,
        "enrollment_date": "2022-09-01",
        "risk_level": "Low",
        "last_assessment": "2024-01-15",
        "recent_grades": [85, 88, 92, 87, 90]
    }

# Create new student
def create_student(student_data: Dict[str, Any]):
    """Create a new student record"""
    
    if api_available:
        try:
            response = requests.post(
                f"{API_URL}/students",
                json=student_data
            )
            return response.json()
        except:
            pass
    
    # Mock response
    return {
        "id": 100,
        "message": "Student created successfully",
        **student_data
    }

# Example usage
print("Fetching students...")
students = get_all_students(limit=5)
print(f"\nFound {len(students)} students:")
for student in students:
    print(f"  - {student['name']} (ID: {student['id']}, Risk: {student['risk_level']})")

print("\nGetting detailed info for student ID 1...")
student_detail = get_student(1)
print(json.dumps(student_detail, indent=2))

print("\nCreating new student...")
new_student = {
    "name": "Alice Wilson",
    "grade_level": 10,
    "enrollment_date": datetime.now().isoformat()
}
created = create_student(new_student)
print(f"Created student with ID: {created.get('id', 'N/A')}")

## 3. Risk Prediction Endpoints

In [ ]:
# Single student prediction
def predict_student_risk(student_id: int):
    """Predict risk level for a single student"""
    
    if api_available:
        try:
            response = requests.post(
                f"{API_URL}/predictions/predict",
                json={"student_id": student_id}
            )
            return response.json()
        except:
            pass
    
    # Mock prediction
    risk_score = np.random.uniform(10, 90)
    return {
        "student_id": student_id,
        "risk_score": risk_score,
        "risk_level": "High" if risk_score > 60 else "Medium" if risk_score > 30 else "Low",
        "confidence": np.random.uniform(75, 95),
        "factors": {
            "academic": np.random.uniform(20, 80),
            "attendance": np.random.uniform(20, 80),
            "behavioral": np.random.uniform(10, 50),
            "engagement": np.random.uniform(30, 90)
        },
        "recommendations": [
            "Schedule meeting with academic counselor",
            "Enroll in tutoring program",
            "Monitor attendance closely"
        ],
        "prediction_date": datetime.now().isoformat()
    }

# Batch prediction
def batch_predict(student_ids: List[int]):
    """Predict risk levels for multiple students"""
    
    if api_available:
        try:
            response = requests.post(
                f"{API_URL}/predictions/batch",
                json={"student_ids": student_ids}
            )
            return response.json()
        except:
            pass
    
    # Mock batch predictions
    predictions = []
    for student_id in student_ids:
        risk_score = np.random.uniform(10, 90)
        predictions.append({
            "student_id": student_id,
            "risk_score": risk_score,
            "risk_level": "High" if risk_score > 60 else "Medium" if risk_score > 30 else "Low",
            "confidence": np.random.uniform(75, 95)
        })
    
    return {"predictions": predictions, "total": len(predictions)}

# Example predictions
print("Single Student Prediction")
print("=" * 40)
prediction = predict_student_risk(1)
print(f"Student ID: {prediction['student_id']}")
print(f"Risk Score: {prediction['risk_score']:.1f}")
print(f"Risk Level: {prediction['risk_level']}")
print(f"Confidence: {prediction['confidence']:.1f}%")
print("\nRisk Factors:")
for factor, score in prediction['factors'].items():
    print(f"  - {factor.capitalize()}: {score:.1f}")
print("\nRecommendations:")
for rec in prediction['recommendations']:
    print(f"  • {rec}")

print("\n" + "=" * 40)
print("Batch Predictions")
print("=" * 40)
batch_results = batch_predict([1, 2, 3, 4, 5])
print(f"Processed {batch_results['total']} predictions:")
for pred in batch_results['predictions']:
    print(f"  Student {pred['student_id']}: {pred['risk_level']} (Score: {pred['risk_score']:.1f})")

## 4. Historical Data and Trends

In [ ]:
# Get historical predictions
def get_prediction_history(student_id: int, days: int = 30):
    """Get historical predictions for a student"""
    
    if api_available:
        try:
            response = requests.get(
                f"{API_URL}/predictions/history/{student_id}",
                params={"days": days}
            )
            return response.json()
        except:
            pass
    
    # Mock historical data
    history = []
    current_date = datetime.now()
    base_score = 45
    
    for i in range(days):
        date = current_date - timedelta(days=days-i)
        score = base_score + np.random.uniform(-5, 5) + (i * 0.3)  # Slight upward trend
        history.append({
            "date": date.isoformat(),
            "risk_score": min(100, max(0, score)),
            "risk_level": "High" if score > 60 else "Medium" if score > 30 else "Low"
        })
    
    return {"student_id": student_id, "history": history}

# Get aggregated statistics
def get_risk_statistics():
    """Get system-wide risk statistics"""
    
    if api_available:
        try:
            response = requests.get(f"{API_URL}/predictions/statistics")
            return response.json()
        except:
            pass
    
    # Mock statistics
    return {
        "total_students": 500,
        "risk_distribution": {
            "high": 75,
            "medium": 150,
            "low": 275
        },
        "average_risk_score": 38.5,
        "trends": {
            "improving": 120,
            "stable": 300,
            "declining": 80
        },
        "interventions_active": 95,
        "success_rate": 72.5
    }

# Visualize historical data
history_data = get_prediction_history(1, 30)
history_df = pd.DataFrame(history_data['history'])
history_df['date'] = pd.to_datetime(history_df['date'])

# Plot trend
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Risk score over time
ax1.plot(history_df['date'], history_df['risk_score'], marker='o', linewidth=2)
ax1.axhline(y=30, color='orange', linestyle='--', alpha=0.5, label='Medium Risk')
ax1.axhline(y=60, color='red', linestyle='--', alpha=0.5, label='High Risk')
ax1.set_xlabel('Date')
ax1.set_ylabel('Risk Score')
ax1.set_title(f'Risk Score Trend - Student {history_data["student_id"]}')
ax1.legend()
ax1.grid(True, alpha=0.3)

# System statistics
stats = get_risk_statistics()
risk_dist = stats['risk_distribution']
colors = ['#FF6B6B', '#FFD93D', '#6BCF7F']
ax2.pie(risk_dist.values(), labels=risk_dist.keys(), colors=colors, autopct='%1.1f%%')
ax2.set_title('System-wide Risk Distribution')

plt.tight_layout()
plt.show()

print("\nSystem Statistics:")
print("=" * 40)
print(f"Total Students: {stats['total_students']}")
print(f"Average Risk Score: {stats['average_risk_score']:.1f}")
print(f"Active Interventions: {stats['interventions_active']}")
print(f"Success Rate: {stats['success_rate']:.1f}%")

## 5. Model Training and Management

In [ ]:
# Trigger model training
def train_model(config: Dict[str, Any] = None):
    """Trigger model training with optional configuration"""
    
    default_config = {
        "epochs": 50,
        "batch_size": 32,
        "learning_rate": 0.001,
        "validation_split": 0.2,
        "early_stopping": True,
        "patience": 5
    }
    
    if config:
        default_config.update(config)
    
    if api_available:
        try:
            response = requests.post(
                f"{API_URL}/training/train",
                json=default_config
            )
            return response.json()
        except:
            pass
    
    # Mock training response
    return {
        "job_id": "train_" + str(int(time.time())),
        "status": "started",
        "config": default_config,
        "estimated_time": "15 minutes",
        "message": "Training job started successfully"
    }

# Check training status
def get_training_status(job_id: str = None):
    """Get status of training job"""
    
    if api_available:
        try:
            url = f"{API_URL}/training/status"
            if job_id:
                url += f"/{job_id}"
            response = requests.get(url)
            return response.json()
        except:
            pass
    
    # Mock status
    return {
        "job_id": job_id or "train_1234567890",
        "status": "completed",
        "progress": 100,
        "current_epoch": 50,
        "total_epochs": 50,
        "metrics": {
            "loss": 0.234,
            "accuracy": 0.875,
            "val_loss": 0.256,
            "val_accuracy": 0.862
        },
        "duration": "14:32",
        "completed_at": datetime.now().isoformat()
    }

# Get model metrics
def get_model_metrics():
    """Get current model performance metrics"""
    
    if api_available:
        try:
            response = requests.get(f"{API_URL}/training/metrics")
            return response.json()
        except:
            pass
    
    # Mock metrics
    return {
        "model_version": "2.1.0",
        "trained_date": "2024-01-15",
        "training_samples": 10000,
        "performance": {
            "accuracy": 87.5,
            "precision": 85.3,
            "recall": 88.7,
            "f1_score": 86.9
        },
        "confusion_matrix": {
            "high_risk": {"correct": 450, "incorrect": 50},
            "medium_risk": {"correct": 780, "incorrect": 120},
            "low_risk": {"correct": 1420, "incorrect": 80}
        },
        "feature_importance": {
            "gpa": 0.35,
            "attendance": 0.28,
            "discipline": 0.15,
            "engagement": 0.12,
            "assignments": 0.10
        }
    }

# Example: Start training
print("Starting Model Training")
print("=" * 40)
training_config = {
    "epochs": 100,
    "batch_size": 64,
    "learning_rate": 0.0005
}
training_job = train_model(training_config)
print(f"Job ID: {training_job['job_id']}")
print(f"Status: {training_job['status']}")
print(f"Estimated Time: {training_job['estimated_time']}")

# Check status
print("\nChecking Training Status...")
status = get_training_status(training_job['job_id'])
print(f"Progress: {status['progress']}%")
print(f"Current Epoch: {status['current_epoch']}/{status['total_epochs']}")
print("\nTraining Metrics:")
for metric, value in status['metrics'].items():
    print(f"  {metric}: {value:.4f}")

# Get model performance
print("\nCurrent Model Performance")
print("=" * 40)
metrics = get_model_metrics()
print(f"Model Version: {metrics['model_version']}")
print(f"Training Samples: {metrics['training_samples']:,}")
print("\nPerformance Metrics:")
for metric, value in metrics['performance'].items():
    print(f"  {metric}: {value:.1f}%")

# Visualize feature importance
features = list(metrics['feature_importance'].keys())
importance = list(metrics['feature_importance'].values())

plt.figure(figsize=(10, 6))
plt.bar(features, importance, color='skyblue')
plt.xlabel('Features')
plt.ylabel('Importance Score')
plt.title('Feature Importance in Risk Prediction Model')
plt.show()

## 6. Asynchronous API Operations

In [ ]:
# Async batch operations
async def async_predict_batch(session, student_ids):
    """Asynchronously predict risk for multiple students"""
    tasks = []
    for student_id in student_ids:
        task = asyncio.create_task(
            async_predict_single(session, student_id)
        )
        tasks.append(task)
    
    results = await asyncio.gather(*tasks)
    return results

async def async_predict_single(session, student_id):
    """Async prediction for single student"""
    if api_available:
        try:
            async with session.post(
                f"{API_URL}/predictions/predict",
                json={"student_id": student_id}
            ) as response:
                return await response.json()
        except:
            pass
    
    # Mock async response
    await asyncio.sleep(0.1)  # Simulate network delay
    risk_score = np.random.uniform(10, 90)
    return {
        "student_id": student_id,
        "risk_score": risk_score,
        "risk_level": "High" if risk_score > 60 else "Medium" if risk_score > 30 else "Low"
    }

# Performance comparison: sync vs async
async def compare_performance():
    """Compare synchronous vs asynchronous API calls"""
    student_ids = list(range(1, 21))  # 20 students
    
    # Synchronous timing
    start_sync = time.time()
    sync_results = []
    for student_id in student_ids:
        result = predict_student_risk(student_id)
        sync_results.append(result)
    sync_time = time.time() - start_sync
    
    # Asynchronous timing
    start_async = time.time()
    async with aiohttp.ClientSession() as session:
        async_results = await async_predict_batch(session, student_ids)
    async_time = time.time() - start_async
    
    return {
        "sync_time": sync_time,
        "async_time": async_time,
        "speedup": sync_time / async_time if async_time > 0 else 0,
        "sync_results": len(sync_results),
        "async_results": len(async_results)
    }

# Run performance comparison
print("Performance Comparison: Sync vs Async")
print("=" * 40)

# Note: In Jupyter, we need to handle async differently
try:
    # Try to get existing event loop
    loop = asyncio.get_event_loop()
except RuntimeError:
    # Create new event loop if none exists
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

# Mock performance comparison (since we can't run async in all environments)
perf_results = {
    "sync_time": 2.5,
    "async_time": 0.8,
    "speedup": 3.125,
    "sync_results": 20,
    "async_results": 20
}

print(f"Synchronous Time: {perf_results['sync_time']:.2f} seconds")
print(f"Asynchronous Time: {perf_results['async_time']:.2f} seconds")
print(f"Speedup: {perf_results['speedup']:.2f}x faster")
print(f"Results processed: {perf_results['async_results']} students")

# Visualize performance difference
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Time comparison
methods = ['Synchronous', 'Asynchronous']
times = [perf_results['sync_time'], perf_results['async_time']]
colors = ['red', 'green']
ax1.bar(methods, times, color=colors, alpha=0.7)
ax1.set_ylabel('Time (seconds)')
ax1.set_title('API Call Performance Comparison')

# Throughput comparison
throughput = [20/perf_results['sync_time'], 20/perf_results['async_time']]
ax2.bar(methods, throughput, color=colors, alpha=0.7)
ax2.set_ylabel('Requests per Second')
ax2.set_title('Throughput Comparison')

plt.tight_layout()
plt.show()

## 7. Error Handling and Retry Logic

In [ ]:
# Robust API client with retry logic
class RobustAPIClient:
    def __init__(self, base_url, max_retries=3, timeout=10):
        self.base_url = base_url
        self.max_retries = max_retries
        self.timeout = timeout
    
    def _make_request(self, method, endpoint, **kwargs):
        """Make HTTP request with retry logic"""
        url = f"{self.base_url}{endpoint}"
        retries = 0
        
        while retries < self.max_retries:
            try:
                response = requests.request(
                    method, url, timeout=self.timeout, **kwargs
                )
                
                # Check for HTTP errors
                if response.status_code >= 500:
                    raise requests.exceptions.HTTPError(
                        f"Server error: {response.status_code}"
                    )
                elif response.status_code >= 400:
                    return {
                        "error": True,
                        "status_code": response.status_code,
                        "message": response.text
                    }
                
                return response.json()
                
            except requests.exceptions.Timeout:
                retries += 1
                if retries >= self.max_retries:
                    return {"error": True, "message": "Request timeout after retries"}
                time.sleep(2 ** retries)  # Exponential backoff
                
            except requests.exceptions.ConnectionError:
                return {"error": True, "message": "Connection error"}
                
            except Exception as e:
                return {"error": True, "message": str(e)}
    
    def get(self, endpoint, params=None):
        return self._make_request("GET", endpoint, params=params)
    
    def post(self, endpoint, json=None):
        return self._make_request("POST", endpoint, json=json)
    
    def put(self, endpoint, json=None):
        return self._make_request("PUT", endpoint, json=json)
    
    def delete(self, endpoint):
        return self._make_request("DELETE", endpoint)

# Example usage with error handling
client = RobustAPIClient(API_URL)

print("Testing Robust API Client")
print("=" * 40)

# Test various scenarios
test_cases = [
    ("Valid request", "/students/1"),
    ("Invalid endpoint", "/invalid/endpoint"),
    ("Server error simulation", "/test/500"),
]

for description, endpoint in test_cases:
    print(f"\n{description}: {endpoint}")
    
    # Simulate response (since we're using mock data)
    if "invalid" in endpoint:
        result = {"error": True, "status_code": 404, "message": "Not found"}
    elif "500" in endpoint:
        result = {"error": True, "message": "Server error after retries"}
    else:
        result = {"id": 1, "name": "John Doe", "status": "success"}
    
    if isinstance(result, dict) and result.get("error"):
        print(f"  ❌ Error: {result.get('message', 'Unknown error')}")
    else:
        print(f"  ✅ Success: {result}")

# Rate limiting handler
class RateLimitedClient(RobustAPIClient):
    def __init__(self, base_url, requests_per_second=10, **kwargs):
        super().__init__(base_url, **kwargs)
        self.requests_per_second = requests_per_second
        self.last_request_time = 0
    
    def _make_request(self, method, endpoint, **kwargs):
        # Enforce rate limiting
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        min_interval = 1.0 / self.requests_per_second
        
        if time_since_last < min_interval:
            sleep_time = min_interval - time_since_last
            time.sleep(sleep_time)
        
        self.last_request_time = time.time()
        return super()._make_request(method, endpoint, **kwargs)

print("\n" + "=" * 40)
print("Rate-Limited Client Example")
print("=" * 40)

rate_limited_client = RateLimitedClient(API_URL, requests_per_second=5)
print("Making 5 requests with rate limiting (5 req/s)...")

start_time = time.time()
for i in range(5):
    # Simulate request
    print(f"  Request {i+1}: Completed")
    time.sleep(0.2)  # Simulate rate limiting

elapsed = time.time() - start_time
print(f"\nTotal time: {elapsed:.2f} seconds")
print(f"Effective rate: {5/elapsed:.2f} requests/second")

## 8. Webhook Integration

In [ ]:
# Webhook configuration
def configure_webhook(webhook_url: str, events: List[str]):
    """Configure webhook for real-time notifications"""
    
    config = {
        "url": webhook_url,
        "events": events,
        "active": True,
        "secret": "your-webhook-secret"
    }
    
    if api_available:
        try:
            response = requests.post(
                f"{API_URL}/webhooks/configure",
                json=config
            )
            return response.json()
        except:
            pass
    
    # Mock response
    return {
        "webhook_id": "wh_123456",
        "url": webhook_url,
        "events": events,
        "status": "configured",
        "message": "Webhook configured successfully"
    }

# Example webhook payload handler
def handle_webhook_payload(payload: Dict[str, Any]):
    """Process incoming webhook payload"""
    
    event_type = payload.get("event")
    data = payload.get("data")
    
    if event_type == "risk_level_changed":
        student_id = data.get("student_id")
        old_level = data.get("old_level")
        new_level = data.get("new_level")
        
        print(f"Alert: Student {student_id} risk changed from {old_level} to {new_level}")
        
        # Trigger appropriate actions
        if new_level == "High":
            print("  → Sending notification to counselor")
            print("  → Scheduling intervention meeting")
            print("  → Creating support plan")
    
    elif event_type == "prediction_completed":
        batch_id = data.get("batch_id")
        total = data.get("total_predictions")
        high_risk_count = data.get("high_risk_count")
        
        print(f"Batch {batch_id} completed: {total} predictions")
        print(f"  High risk students identified: {high_risk_count}")
    
    elif event_type == "model_trained":
        model_version = data.get("version")
        accuracy = data.get("accuracy")
        
        print(f"New model trained: v{model_version}")
        print(f"  Accuracy: {accuracy:.2%}")

# Configure webhook
print("Webhook Configuration")
print("=" * 40)

webhook_events = [
    "risk_level_changed",
    "prediction_completed",
    "model_trained",
    "intervention_needed"
]

webhook_result = configure_webhook(
    "https://your-app.com/webhooks/edupulse",
    webhook_events
)

print(f"Webhook ID: {webhook_result['webhook_id']}")
print(f"Status: {webhook_result['status']}")
print(f"Configured events: {', '.join(webhook_result['events'])}")

# Simulate webhook events
print("\nSimulating Webhook Events")
print("=" * 40)

sample_payloads = [
    {
        "event": "risk_level_changed",
        "timestamp": datetime.now().isoformat(),
        "data": {
            "student_id": 123,
            "old_level": "Medium",
            "new_level": "High",
            "risk_score": 72.5
        }
    },
    {
        "event": "prediction_completed",
        "timestamp": datetime.now().isoformat(),
        "data": {
            "batch_id": "batch_456",
            "total_predictions": 50,
            "high_risk_count": 8,
            "processing_time": 2.34
        }
    }
]

for payload in sample_payloads:
    print(f"\nEvent: {payload['event']}")
    handle_webhook_payload(payload)

## 9. API Client SDK Example

In [ ]:
# Complete SDK implementation
class EduPulseSDK:
    """Official EduPulse API Python SDK"""
    
    def __init__(self, base_url=None, api_key=None):
        self.base_url = base_url or "http://localhost:8000/api/v1"
        self.api_key = api_key
        self.session = requests.Session()
        if api_key:
            self.session.headers.update({"X-API-Key": api_key})
    
    # Student Management
    def get_student(self, student_id):
        return self._get(f"/students/{student_id}")
    
    def list_students(self, limit=100, offset=0, risk_level=None):
        params = {"limit": limit, "offset": offset}
        if risk_level:
            params["risk_level"] = risk_level
        return self._get("/students", params=params)
    
    def create_student(self, student_data):
        return self._post("/students", json=student_data)
    
    def update_student(self, student_id, updates):
        return self._put(f"/students/{student_id}", json=updates)
    
    # Predictions
    def predict_risk(self, student_id):
        return self._post("/predictions/predict", json={"student_id": student_id})
    
    def batch_predict(self, student_ids):
        return self._post("/predictions/batch", json={"student_ids": student_ids})
    
    def get_prediction_history(self, student_id, days=30):
        return self._get(f"/predictions/history/{student_id}", params={"days": days})
    
    # Training
    def train_model(self, config=None):
        return self._post("/training/train", json=config or {})
    
    def get_training_status(self, job_id=None):
        endpoint = f"/training/status/{job_id}" if job_id else "/training/status"
        return self._get(endpoint)
    
    def get_model_metrics(self):
        return self._get("/training/metrics")
    
    # Helper methods
    def _get(self, endpoint, params=None):
        return self._request("GET", endpoint, params=params)
    
    def _post(self, endpoint, json=None):
        return self._request("POST", endpoint, json=json)
    
    def _put(self, endpoint, json=None):
        return self._request("PUT", endpoint, json=json)
    
    def _delete(self, endpoint):
        return self._request("DELETE", endpoint)
    
    def _request(self, method, endpoint, **kwargs):
        url = f"{self.base_url}{endpoint}"
        try:
            response = self.session.request(method, url, **kwargs)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            # Return mock data for demonstration
            return self._mock_response(endpoint, method)
    
    def _mock_response(self, endpoint, method):
        """Generate mock responses for demonstration"""
        if "students" in endpoint and method == "GET":
            return [{"id": 1, "name": "John Doe", "risk_level": "Low"}]
        elif "predict" in endpoint:
            return {"risk_score": 45.2, "risk_level": "Medium"}
        elif "metrics" in endpoint:
            return {"accuracy": 87.5, "precision": 85.3}
        return {"status": "success"}

# SDK Usage Examples
print("EduPulse SDK Examples")
print("=" * 50)

# Initialize SDK
sdk = EduPulseSDK(api_key="your-api-key-here")

# Example 1: Get high-risk students
print("\n1. Fetching high-risk students...")
high_risk = sdk.list_students(limit=10, risk_level="High")
print(f"   Found {len(high_risk)} high-risk students")

# Example 2: Predict risk for a student
print("\n2. Predicting risk for student 123...")
prediction = sdk.predict_risk(123)
print(f"   Risk Level: {prediction.get('risk_level', 'Unknown')}")
print(f"   Risk Score: {prediction.get('risk_score', 0):.1f}")

# Example 3: Batch predictions
print("\n3. Running batch predictions...")
batch_results = sdk.batch_predict([1, 2, 3, 4, 5])
print(f"   Processed {len(batch_results)} predictions")

# Example 4: Get model metrics
print("\n4. Fetching model performance metrics...")
metrics = sdk.get_model_metrics()
print(f"   Model Accuracy: {metrics.get('accuracy', 0):.1f}%")
print(f"   Model Precision: {metrics.get('precision', 0):.1f}%")

# Example 5: Train new model
print("\n5. Starting model training...")
training_job = sdk.train_model({"epochs": 100, "batch_size": 32})
print(f"   Training job started: {training_job.get('status', 'Unknown')}")

print("\n" + "=" * 50)
print("SDK integration complete!")

## Conclusion

This notebook has demonstrated comprehensive usage of the EduPulse API including:

### Key Features Covered
1. **Basic Operations**: Health checks, student management, predictions
2. **Batch Processing**: Efficient handling of multiple requests
3. **Model Management**: Training, monitoring, and metrics
4. **Advanced Patterns**: Async operations, error handling, rate limiting
5. **Integration**: Webhooks and SDK implementation

### Best Practices
- Always implement error handling and retry logic
- Use batch endpoints for processing multiple items
- Implement rate limiting to avoid overwhelming the API
- Use async operations for better performance
- Monitor API health and metrics regularly

### Next Steps
1. Implement authentication and authorization
2. Set up monitoring and logging
3. Create API documentation with OpenAPI/Swagger
4. Implement caching for frequently accessed data
5. Set up automated testing for API endpoints

For production use, ensure proper API key management, implement SSL/TLS, and follow security best practices.